In [1]:
import tkinter as tk
import re


class Parser:
    def __init__(self):
        self.tokens = []

    def tokenize_input(self, input_string):
        """
        Tokenizes the input string based on grammar rules.
        """
        self.tokens = re.findall(r'\d+|[-+*/()]', input_string)

    def check_syntax(self):
        """
        Checks the syntax of the arithmetic expression.
        """
        stack = []
        for token in self.tokens:
            if token == '(':
                stack.append(token)
            elif token == ')':
                if not stack or stack.pop() != '(':
                    return False
          
        return len(stack) == 0

    def parse_expression(self):
        """
        Parses the arithmetic expression recursively.
        """
        self.current_token_index = 0
        result = self.parse_term()
        while self.current_token_index < len(self.tokens):
            if self.tokens[self.current_token_index] in {'+', '-'}:
                operator = self.tokens[self.current_token_index]
                self.current_token_index += 1
                term = self.parse_term()
                if operator == '+':
                    result += term
                else:
                    result -= term
            else:
                break
        return result

    def parse_term(self):
        """
        Parses a term (number or subexpression).
        """
        result = self.parse_factor()
        while self.current_token_index < len(self.tokens):
            if self.tokens[self.current_token_index] in {'*', '/'}:
                operator = self.tokens[self.current_token_index]
                self.current_token_index += 1
                factor = self.parse_factor()
                if operator == '*':
                    result *= factor
                else:
                    if factor != 0:
                        result /= factor
                    else:
                        raise ValueError("Division by zero")
            else:
                break
        return result

    def parse_factor(self):
     
        if self.tokens[self.current_token_index] == '(':
            self.current_token_index += 1
            result = self.parse_expression()
            if self.tokens[self.current_token_index] == ')':
                self.current_token_index += 1
                return result
            else:
                raise ValueError("Mismatched parentheses")
        elif self.tokens[self.current_token_index].isdigit():
            result = int(self.tokens[self.current_token_index])
            self.current_token_index += 1
            return result
        else:
            raise ValueError("Invalid token")

    def parse_input(self, tokens):
       
        self.tokens = tokens
        self.current_token_index = 0
        return self.parse_expression()






In [2]:
class ArithmeticExpressionParserGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Arithmetic Expression Parser")

        # Text box for code input
        self.code_textbox = tk.Text(self.root, height=10, width=50)
        self.code_textbox.pack()

        # Check syntax button
        self.check_syntax_button = tk.Button(self.root, text="Check Syntax", command=self.check_syntax)
        self.check_syntax_button.pack()

        # Execute button
        self.execute_button = tk.Button(self.root, text="Execute", command=self.execute_code)
        self.execute_button.pack()

        # Error logger
        self.error_logger = tk.Text(self.root, height=5, width=50)
        self.error_logger.pack()

        # Parser instance
        self.parser = Parser()

    def check_syntax(self):
        # Clear previous error messages
        self.error_logger.delete('1.0', tk.END)

        try:
            input_code = self.code_textbox.get("1.0", tk.END)
            self.parser.tokenize_input(input_code)
            if self.parser.check_syntax():
                self.error_logger.insert(tk.END, "Syntax check passed: No syntax errors found.")
            else:
                self.error_logger.insert(tk.END, "Syntax Error: Mismatched parentheses.")
        except ValueError as e:
            self.error_logger.insert(tk.END, f"Syntax Error: {str(e)}")

    def execute_code(self):
        # Clear previous result and error messages
        self.error_logger.delete('1.0', tk.END)

        try:
            input_code = self.code_textbox.get("1.0", tk.END)
            self.parser.tokenize_input(input_code)
            result = self.parser.parse_input(self.parser.tokens)
            self.error_logger.insert(tk.END, f"Result: {result}")
        except ValueError as e:
            self.error_logger.insert(tk.END, f"Error: {str(e)}")


In [3]:
if __name__ == "__main__":
    root = tk.Tk()
    app = ArithmeticExpressionParserGUI(root)
    root.mainloop()
